In [13]:
!pip install tpot -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
    100% |████████████████████████████████| 92kB 889kB/s ta 0:00:011
    100% |████████████████████████████████| 112kB 313kB/s ta 0:00:01
    100% |████████████████████████████████| 7.2MB 2.2MB/s ta 0:00:011
    100% |████████████████████████████████| 10.4MB 1.2MB/s ta 0:00:011   72% |███████████████████████▎        | 7.5MB 9.6MB/s eta 0:00:01    78% |█████████████████████████▏      | 8.2MB 6.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for stopit ... done
  Stored in directory: /Users/yangkaixiang/Library/Caches/pip/wheels/9a/ad/bc/c99681174e9db3200ad8b7cce37dfe5cf213dc8b21cc739ff2
Successfully built stopit
  Found existing installation: scikit-learn 0.20.1
    Uninstalling scikit-learn-0.20.1:
      Successfully uninstalled scikit-learn-0.20.1
  Found existing installation: pandas 0.23.4
    Uninstalling pandas-0.23.4:
      Successfully uninstalled pandas-0.23.4


In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.feature_extraction import DictVectorizer

# 数据加载
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [2]:
# 数据探索
# 查看train_data信息
#pd.set_option('display.max_columns', None) #显示所有列
print('查看数据信息：列名、非空个数、类型等')
print(train_data.info())
print('-'*30)
print('查看数据摘要')
print(train_data.describe())
print('-'*30)
print('查看离散数据分布')
print(train_data.describe(include=['O']))
print('-'*30)
print('查看前5条数据')
print(train_data.head())
print('-'*30)
print('查看后5条数据')
print(train_data.tail())

查看数据信息：列名、非空个数、类型等
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
------------------------------
查看数据摘要
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642

In [3]:
# 使用平均年龄来填充年龄中的nan值
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)
# 使用票价的均值填充票价中的nan值
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)

print(train_data['Embarked'].value_counts())
# 使用登录最多的港口来填充登录港口的nan值
train_data['Embarked'].fillna('S', inplace=True)
test_data['Embarked'].fillna('S',inplace=True)
# 特征选择
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
test_features = test_data[features]
print('特征值')
print(train_features)

dvec=DictVectorizer(sparse=False)
train_features=dvec.fit_transform(train_features.to_dict(orient='record'))
print(dvec.feature_names_)

S    644
C    168
Q     77
Name: Embarked, dtype: int64
特征值
     Pclass     Sex        Age  SibSp  Parch     Fare Embarked
0         3    male  22.000000      1      0   7.2500        S
1         1  female  38.000000      1      0  71.2833        C
2         3  female  26.000000      0      0   7.9250        S
3         1  female  35.000000      1      0  53.1000        S
4         3    male  35.000000      0      0   8.0500        S
..      ...     ...        ...    ...    ...      ...      ...
886       2    male  27.000000      0      0  13.0000        S
887       1  female  19.000000      0      0  30.0000        S
888       3  female  29.699118      1      2  23.4500        S
889       1    male  26.000000      0      0  30.0000        C
890       3    male  32.000000      0      0   7.7500        Q

[891 rows x 7 columns]
['Age', 'Embarked=C', 'Embarked=Q', 'Embarked=S', 'Fare', 'Parch', 'Pclass', 'Sex=female', 'Sex=male', 'SibSp']


/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [4]:
# 构造ID3决策树
clf = DecisionTreeClassifier(criterion='entropy')
# 决策树训练
clf.fit(train_features, train_labels)

test_features=dvec.transform(test_features.to_dict(orient='record'))
# 决策树预测
pred_labels = clf.predict(test_features)

# 得到决策树准确率(基于训练集)
acc_decision_tree = round(clf.score(train_features, train_labels), 6)
print(u'score准确率为 %.4lf' % acc_decision_tree)

# 使用K折交叉验证 统计决策树准确率
print(u'cross_val_score准确率为 %.4lf' % np.mean(cross_val_score(clf, train_features, train_labels, cv=10)))

score准确率为 0.9820
cross_val_score准确率为 0.7801


/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [5]:
from tpot import TPOTClassifier  # tpot
#TPOT模型
model = TPOTClassifier(
    generations=5,   # 运行管道优化过程的迭代次数
    population_size=20,   # 在遗传进化中每一代要保留的个体数量
    verbosity=2,   # TPOT运行时能传递多少信息
)

In [6]:
model.fit(train_features, train_labels)

/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



Generation 1 - Current best internal CV score: 0.8204381394764925
Generation 2 - Current best internal CV score: 0.8283095850856819
Generation 3 - Current best internal CV score: 0.8283095850856819
Generation 4 - Current best internal CV score: 0.8294017952419811
Generation 5 - Current best internal CV score: 0.8294017952419811
Best pipeline: ExtraTreesClassifier(VarianceThreshold(MLPClassifier(input_matrix, alpha=0.1, learning_rate_init=0.1), threshold=0.0001), bootstrap=True, criterion=entropy, max_features=0.25, min_samples_leaf=1, min_samples_split=8, n_estimators=100)


TPOTClassifier(generations=5,
               log_file=<ipykernel.iostream.OutStream object at 0x10dc085c0>,
               population_size=20, verbosity=2)

In [7]:
model.score(train_features, train_labels)

0.8933782267115601

In [8]:
model.export('titanic_tpot_pipeline.py')